In [1]:
import random

In [310]:
def pairings(fighters):
    all_keys = list(fighters.keys())
    if len(all_keys)%2 != 0:
        raise ValueError('Must have even number of fighters')
    fights_num = int(len(all_keys)/2)
    random.shuffle(all_keys)
    all_pairs = []
    for i in range(fights_num):
        all_pairs.append((all_keys[2*i], all_keys[2*i+1]))
    return all_pairs

In [311]:
fighters = {}
lost_fighters = []
NUM_F = 40
START_HP = 12

for i in range(NUM_F):
    fighters[i] = START_HP

In [312]:
def clean(fighters, lost_keys):
    fighters_to_remove = []
    for f in fighters.keys():
        if fighters[f] <=0:
            fighters_to_remove.append(f)
    for f in fighters_to_remove:
        fighters.pop(f)
        lost_keys.append(f)
    return fighters, lost_keys

def conduct_round(fighters, lost_keys, v):
    if len(fighters.keys())%2 !=0:
        fighters[lost_keys[-1]] = 3
        lost_keys = lost_keys[:-1]
    pairs = pairings(fighters)
    for p in pairs:
        before_1 = fighters[p[0]]
        before_2 = fighters[p[1]]
        fighters[p[0]] = fighters[p[0]] - random.randint(1,3)
        fighters[p[1]] = fighters[p[1]] - random.randint(1,3)    
        if(v >= 2):
            print(p[0], ':', before_1, '-->', fighters[p[0]])
            print(p[1], ':', before_2, '-->',  fighters[p[1]])
    return clean(fighters, lost_keys)

In [313]:
# Initial setup
def simulate(NUM_F = 10, v=1):
    fighters = {}
    lost_fighters = []
    START_HP = 12
    for i in range(NUM_F):
        fighters[i] = START_HP

    # rounds
    total_fights = 0
    rounds = 0
    while (len(fighters.keys()) > 6):
        rounds += 1
        fights_in_round = int((len(fighters.keys()) + 1)/2)
        total_fights += fights_in_round
        fighters, lost_fighters = conduct_round(fighters, lost_fighters,v)
        if v >= 1:
            print("Round {}".format(rounds))
            print("{} fights in round, {} fighters out, {} fighters left.".format(fights_in_round, len(lost_fighters), len(fighters.keys())))
            print("Total fights {}, time {}".format(total_fights, total_fights*2 + 5*rounds))
            print("{} total points left")
        if v >= 2:
            print(fighters)
            print(lost_fighters)
    return rounds, total_fights

In [324]:
import numpy as np

rounds = []
fights = []

for i in range (100):
    r, f = simulate(70,0)
    #print(r,f)
    rounds.append(r)
    fights.append(f)
print('Rounds: ', np.min(rounds), np.mean(rounds), np.max(rounds))
print('fights: ', np.min(fights), np.mean(fights), np.max(fights))

Rounds:  7 7.8 8
fights:  211 221.43 231


In [319]:
def tournament_time(rounds, fights, grounds):
    return (5*rounds + fights*3/grounds)/60

In [325]:
tournament_time(8, 230, 2)

6.416666666666667

# Swiss pairings

In [218]:
class Fighter:
    
    def __init__(self, name, hp=12):
        self.name = name
        self.hp = hp
        self.enemies = {}#{self.name:9999}
    
    def fight(self, other, hp_lost):
        if other.name in self.enemies.keys():
            self.enemies[other.name] += 1
        else:
            self.enemies[other.name] = 1
        self.hp -= hp_lost
        
    def played(self, other):
        if other.name in self.enemies.keys():
            return self.enemies[other.name]
        else:
            return 0

    def normalize_played(self, others):
        if len(self.enemies.keys()) < len(others):
            return
        m = 99
        for o in others:
            if o.name not in self.enemies.keys():
                return
            else:
                m = min(m, self.enemies[o.name])
        if m == 0:
            return
        # If we reached here it means that all others played at least 1 time
        # Then we substract 1 from all of them to nullify at least one of them
        print(self.name + " played with all at least {} times".format(m))
        for o in others:
            self.enemies[o.name] -= m
        
    def __repr__(self):
        return (self.name + ' ' + str(self.hp))

In [405]:
def fight(f1, f2, result):
    f1.fight(f2, result[0])
    f2.fight(f1, result[1])

def playerStandings(players):
    """Returns a list of the players and their win records, sorted by score.

    The first entry in the list should be the player in first place, or a player
    tied for first place if there is currently a tie.

    Args:
        None
    Returns:
      A list of tuples, each of which contains (id, name, wins, losses, draws, matches, score):
        (0)id: the player's unique id (assigned by the database)
        (1)name: the player's full name (as registered)
        (6)score: the score of this player
    """
    def hp(fighter):
        return fighter.hp
    
    return sorted(players, key=hp)

def alreadyPlayed(player1, player2):
    """Have these two players already played in this tour?

    Args:
        player1: player's unique id#
        player2: player's unique id#
        tour_id: tour's unique id#
    Returns:
        A boolean indicating if this would be a rematch
    """
    return player1.played(player2) > 0 or player2.played(player1) > 0

def normalize_all_played(fighters):
    for f in fighters:
        f.normalize_played([other for other in fighters if other.name != f.name])

def swissPairings(players):
    """Returns a list of pairs of players for the next round of a match in this tour.

    Assuming that there are an even number of players registered, each player
    appears exactly once in the pairings.  Each player is paired with another
    player with an equal or nearly-equal win record, that is, a player adjacent
    to him or her in the standings.  Rematches are not allowed, so all pairings are new.

    Returns:
      A list of tuples, each of which contains (id1, name1, id2, name2)
        id1: the first player's unique id
        name1: the first player's name
        id2: the second player's unique id
        name2: the second player's name
    """
    standings = playerStandings(players)
    pairings = []

    for i in range(0, len(standings), 2):
        for j in range(i+1, len(standings)):
            if not alreadyPlayed(standings[i], standings[j]):
                # good pair found, swap into place then break the inner loop
                while j > i + 1:
                    standings[j-1], standings[j] = standings[j], standings[j-1]
                    j -= 1
                break
        pairings.append((standings[i], standings[i+1]))
    return pairings

def remove(fighters, outs, v):
    new_outs = []
    m = 3
    for f in fighters:
        if f.hp <= 0:
            new_outs.append(f)
        else:
            m = min(m, f.hp)
            
    if len(new_outs)%2 != 0:
        lucky = random.choice(new_outs)
        if v >= 1:
            print ('Lucky {}'.format(lucky.name))
        lucky.hp = m
        new_outs.remove(lucky)
        
    for f in new_outs:
        fighters.remove(f)
        
    outs += new_outs

In [404]:
def conduct_round_swiss(fighters, outs, v):
    pairs = swissPairings(fighters)
    #print(pairs)
    for p in pairs:
        fight(p[0], p[1], (random.randint(1,3), random.randint(1,3)))
    normalize_all_played(fighters)
    remove(fighters, outs, v)

In [387]:
fighters = [Fighter(str(i)) for i in range(10)]

In [393]:
outs = []
conduct_round_swiss(fighters, outs)
fighters

[(7 1, 9 1), (1 3, 2 3), (3 3, 4 4), (0 4, 5 6)]


[0 2, 1 2, 2 1, 3 1, 4 2, 5 3]

In [300]:
outs

[2 0, 20 -1, 27 -4, 35 -3]

In [445]:
def simulate_swiss(NUM_F = 10, v=1):
    fighters = [Fighter(str(i)) for i in range(NUM_F)]
    outs = []

    # rounds
    total_fights = 0
    rounds = 0
    while (len(fighters) > 6):
        rounds += 1
        fights_in_round = int(len(fighters)/2)
        total_fights += fights_in_round
        conduct_round_swiss(fighters, outs, v)
        if v >= 1:
            print("Round {}".format(rounds))
            print("{} fights in round, {} fighters out, {} fighters left.".format(fights_in_round, len(outs), len(fighters)))
            print("Total fights {}".format(total_fights))
           # print("{} total points left")
        if v >= 2:
            print(fighters)
            print(outs)
    return rounds, total_fights

In [455]:
rounds = []
fights = []
for i in range (100):
    r, f = simulate_swiss(70,0)
    #print(r,f)
    rounds.append(r)
    fights.append(f)
print('Rounds: ', np.min(rounds), np.mean(rounds), np.max(rounds))
print('fights: ', np.min(fights), np.mean(fights), np.max(fights))
print(tournament_time(np.max(rounds), np.max(fights), 2))

49 played with all at least 1 times
Rounds:  7 7.78 8
fights:  211 220.72 229
6.39166666667


# Google sheets

In [3]:
import httplib2
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials as SAC

CREDENTIALS_FILE = 'mws-tournament-fa7e5f4c8445.json'

credentials = SAC.from_json_keyfile_name(CREDENTIALS_FILE, ['https://www.googleapis.com/auth/spreadsheets',
                                                              'https://www.googleapis.com/auth/drive'])
httpAuth = credentials.authorize(httplib2.Http())
service = apiclient.discovery.build('sheets', 'v4', http=httpAuth)

spreadsheet = service.spreadsheets().create(body = {
    'properties': {'title': 'Mws-sheets', 'locale': 'ru_RU'},
    'sheets': [{'properties': {'sheetType': 'GRID',
                               'sheetId': 0,
                               'title': 'Round_1',
                               'gridProperties': {'rowCount': 23, 'columnCount': 15}}}]
}).execute()


In [4]:
spreadsheet

{'properties': {'autoRecalc': 'ON_CHANGE',
  'defaultFormat': {'backgroundColor': {'blue': 1, 'green': 1, 'red': 1},
   'padding': {'bottom': 2, 'left': 3, 'right': 3, 'top': 2},
   'textFormat': {'bold': False,
    'fontFamily': 'arial,sans,sans-serif',
    'fontSize': 10,
    'foregroundColor': {},
    'italic': False,
    'strikethrough': False,
    'underline': False},
   'verticalAlignment': 'BOTTOM',
   'wrapStrategy': 'OVERFLOW_CELL'},
  'locale': 'ru_RU',
  'timeZone': 'Etc/GMT',
  'title': 'Mws-sheets'},
 'sheets': [{'properties': {'gridProperties': {'columnCount': 15,
     'rowCount': 23},
    'index': 0,
    'sheetId': 0,
    'sheetType': 'GRID',
    'title': 'Round_1'}}],
 'spreadsheetId': '1P4cP-V8FWYa7Jwu1IEEWSQ1dmMG9vu6El-x54xspifQ',
 'spreadsheetUrl': 'https://docs.google.com/spreadsheets/d/1P4cP-V8FWYa7Jwu1IEEWSQ1dmMG9vu6El-x54xspifQ/edit'}

In [9]:
# Share for manual updates

driveService = apiclient.discovery.build('drive', 'v3', http = httpAuth)
shareRes = driveService.permissions().create(
    fileId = spreadsheet['spreadsheetId'],
    body = {'type': 'user', 'role': 'writer', 'emailAddress': 'alexey.trekin@gmail.com'},
    fields = 'id'
).execute()

In [12]:
# set column width
request = [] 
for col in [2,3,4,5,7,9,10,11,12]:
    request.append({
          "updateDimensionProperties": {
          "range": {
          "sheetId": 0,
          "dimension": "COLUMNS",  # COLUMNS - потому что столбец
          "startIndex": col,         # Столбцы нумеруются с нуля
          "endIndex": col+1            # startIndex берётся включительно, endIndex - НЕ включительно,
                                   # т.е. размер будет применён к столбцам в диапазоне [0,1), т.е. только к столбцу A
          },
          "properties": {
          "pixelSize": 35     # размер в пикселях
          },
          "fields": "pixelSize"  # нужно задать только pixelSize и не трогать другие параметры столбца
          }
    })

results = service.spreadsheets().batchUpdate(spreadsheetId = spreadsheet['spreadsheetId'], 
                                             body = {"requests": request}).execute()

In [14]:
# merge some cells
request = [
{'mergeCells': {'range': {'sheetId': 0,
                          'startRowIndex': 0,
                          'endRowIndex': 1,
                          'startColumnIndex': 1,
                          'endColumnIndex': 7},
                'mergeType': 'MERGE_ALL'}},
{'mergeCells': {'range': {'sheetId': 0,
                          'startRowIndex': 0,
                          'endRowIndex': 1,
                          'startColumnIndex': 8,
                          'endColumnIndex': 14},
                'mergeType': 'MERGE_ALL'}}]

results = service.spreadsheets().batchUpdate(spreadsheetId = spreadsheet['spreadsheetId'], 
                                             body = {"requests": request}).execute()

In [39]:
# Enter static data
data_request  = {
 "valueInputOption": "USER_ENTERED",
    "data": [
        {"range": "Round_1!B1:B1",
         "majorDimension": "ROWS",     # сначала заполнять ряды, затем столбцы (т.е. самые внутренние списки в values - это ряды)
         "values": [["1 ристалище"]]},

        {"range": "Round_1!I1:I1",
         "majorDimension": "COLUMNS",  # сначала заполнять столбцы, затем ряды (т.е. самые внутренние списки в values - это столбцы)
         "values": [["2 ристалище"]]},
        
        {"range": "Round_1!B2:N2",
         "majorDimension": "ROWS",     # сначала заполнять ряды, затем столбцы (т.е. самые внутренние списки в values - это ряды)
         "values": [["Фамилия", "HP", "Бой", "Бой","HP","Фамилия", "", "Фамилия", "HP", "Бой", "Бой","HP","Фамилия" ]]},
    ]}

results = service.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheet['spreadsheetId'], 
                                             body = data_request).execute()

In [40]:
def get_position(round_number, area, pair_num):
    # 1-based area
    if area == 1:
        columns = ('B', 'G')
    elif area == 2:
        columns = ('I', 'N')
    else: 
        raise ValueError("Only 0 and 1 are valid areas")
    sheet = 'Round_' + str(round_number)
    row = pair_num + 2 # heading
    return '{sheet}!{begin}{row}:{end}{row}'.format(sheet=sheet, row=row, begin = columns[0], end = columns[1])

def get_data(red_fighter, blue_fighter):
    return [red_fighter.name, red_fighter.hp, '', '', blue_fighter.hp, blue_fighter.name]
    
def write_pair(spreadsheetId, round_number, area, pair_num, red_fighter, blue_fighter):
    position = get_position(round_number, area, pair_num)
    pair_data = get_data(red_fighter, blue_fighter)
    data_request  = {
     "valueInputOption": "USER_ENTERED",
     "data": [
        {"range": position,
         "majorDimension": "ROWS",     # сначала заполнять ряды, затем столбцы (т.е. самые внутренние списки в values - это ряды)
         "values": [pair_data]}
    ]}
    results = service.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheetId, 
                                             body = data_request).execute()

In [41]:
get_position(1, 1, 1)

'Round_1!B3:G3'

In [49]:
from fighter import Fighter

blue = Fighter("Loki", 10)
red = Fighter("Thor", 12)

write_pair(spreadsheet['spreadsheetId'], 1, 2, 2, red, blue)

In [47]:
# Color
requests = [
    {
      "repeatCell": {
        "range": {
          "sheetId": 0,
          "startRowIndex": 2,
          "endRowIndex": 23,
          "startColumnIndex": 2,
          "endColumnIndex": 5
        },
        "cell": {
          "userEnteredFormat": {
            "backgroundColor": {
              "red": 0.8,
              "green": 0.8,
              "blue": 1.0
            }
          }
        },
        "fields": "userEnteredFormat(backgroundColor)"
      }
    }
  ]


result = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet['spreadsheetId'], body={'requests': request}).execute()


HttpError: <HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/1P4cP-V8FWYa7Jwu1IEEWSQ1dmMG9vu6El-x54xspifQ:batchUpdate?alt=json returned "Invalid JSON payload received. Unknown name "startIndex" at 'requests[0].update_cells.range': Cannot find field.
Invalid JSON payload received. Unknown name "endIndex" at 'requests[0].update_cells.range': Cannot find field.
Invalid JSON payload received. Unknown name "textStyle" at 'requests[0].update_cells': Cannot find field.". Details: "[{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'field': 'requests[0].update_cells.range', 'description': 'Invalid JSON payload received. Unknown name "startIndex" at \'requests[0].update_cells.range\': Cannot find field.'}, {'field': 'requests[0].update_cells.range', 'description': 'Invalid JSON payload received. Unknown name "endIndex" at \'requests[0].update_cells.range\': Cannot find field.'}, {'field': 'requests[0].update_cells', 'description': 'Invalid JS

# Read data back from the sheet

In [58]:
import pprint
pp = pprint.PrettyPrinter()

request = service.spreadsheets().get(spreadsheetId=spreadsheet['spreadsheetId'], ranges='B3:G21', includeGridData=True)
response = request.execute()
pp.pprint(response)

{'properties': {'autoRecalc': 'ON_CHANGE',
                'defaultFormat': {'backgroundColor': {'blue': 1,
                                                      'green': 1,
                                                      'red': 1},
                                  'padding': {'bottom': 2,
                                              'left': 3,
                                              'right': 3,
                                              'top': 2},
                                  'textFormat': {'bold': False,
                                                 'fontFamily': 'arial,sans,sans-serif',
                                                 'fontSize': 10,
                                                 'foregroundColor': {},
                                                 'italic': False,
                                                 'strikethrough': False,
                                                 'underline': False},
                                